К текущему моменту мы научились обучать простейшую линейную регрессию: причем научились пользоваться как готовыми методами из sklearn, так и реализовывать поиск решения оптимальных коэффициентов самостоятельно с помощью градиентного спуска.

На последних занятиях мы узнали, что на практике часто приходится прибегать к некоторым хитростям, чтобы оценить, действительно ли модель отлично справляется со своей главной задачей - делать хорошие предсказания на новых данных в будущем, или, как еще говорят, в проде - после запуска.

Для этого было введено понятие так называемой обобщающей способности модели. Оно помогает нам оценить, насколько наша модель подогналась под увиденные данные и насколько она окажется хороша на новых данных.

Обобщающую способность мы научились оценивать с помощью метода Кросс-Валидации.



Построим для старой задачи с предсказанием длительности поездки в такси 2 модели линейной регрессии. Для каждой из них замерим качество на кросс-валидации и на тесте, заранее его отложив.

Считается, что если для K-Fold Кросс-Валидации брать большие значения K (то есть большое количество сплитов для разбиения тренировочной выборки на тренировку-валидацию), то оценка среднеквадратической ошибки будет получаться более справедливой. Так, например, когда K оказывается максимальным, а именно равным количеству объектов в выборке (каждый раз в валидации у нас 1 новый объект), то такой метод Кросс-Валидации еще называют LeaveOneOut.

Главная причина, почему мало где его используют, состоит в дороговизне такого метода. Например, в данной задаче в TAXI_train.csv лежит почти 1,2 млн. объектов. Тогда, чтобы замерить качество на LeaveOneOut валидации, нам пришлось бы обучить 1,2 млн. различных версий одной модели, что, конечно, не вписыватеся в рамки адекватного времени и прочих ресурсов.

Поэтому выберем компромиссное значение в виде 20 фолдов для K-Fold Кросс-Валидации.

Внимание! Для каждой из 2 моделей очевидно было бы справедливым замерять качество на тех же самых объектах, поэтому сплиты лучше зафиксировать сразу.

P.S. Оптимизировать будем MSLE метрику. И сравнивать модели тоже будем по ней будем. Как из домашнего задания про метрики, только без корня! :)

$$\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2$$


### Модель №1 Для начала посчитаем ошибку на Кросс-Валидации и Тесте для нашей самой базовой модели до вычленения каких-либо признаков, а просто взяв все вещественные колонки.

In [1]:
import pandas as pd

In [4]:
initial_data = pd.read_csv('../data/cross_validation/initial_data.csv', index_col='id')

initial_cols = ['vendor_id', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'trip_duration']

initial_data = initial_data[initial_cols]

### ТрюК!

In [5]:
initial_data.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,455.0
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,663.0
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,2124.0
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,429.0
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,435.0


In [9]:
# logarithmed data:


np.log1p(455)

6.1224928095143865

In [11]:
# reversed from log to initial number:

np.e ** 6.1224928095143865 - 1


455.0

In [6]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

initial_data = initial_data.assign(log_trip_duration=np.log1p(initial_data['trip_duration']))
initial_data = initial_data.drop('trip_duration', axis=1)


initial_data.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,log_trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,6.122493
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,6.498282
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,7.661527
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,6.063785
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,6.077642


### Объяснение:

Пусть имеем модель $a(x)$, обученную на MSE от $y$:
$$
\text{MSE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(y_i - a(x_i)\big)^2 \rightarrow min
$$

Также представим модель $a^*(x)$, обученную на MSE от $\log{(y + 1)}$:
$$
\text{MSE}(X, \log{(y+1)}, a^*) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - a^*(x_i)\big)^2 \rightarrow min
$$

Так как вторая модель старается аппроксимироваться ответы в виде $a^*(x) \approx \log{(y+1)}$, то для того, чтобы получить изначальные таргеты, необходимо выходы $a^*(x)$ проэкспоненцировать в виде следующего отношения: $a(x) = e^{a^*(x)} - 1$, то есть $a^*(x) = \log{(a(x) + 1)}$, где $a^*(x)$ дает логарифмические выходы, а $a(x)$ - изначальные. Подставим данное соотношение в MSE выше, получим:

$$
\text{MSE}(X, \log{(y+1)}, \log{(a(x) + 1)}) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

А это в точности MSLE от $y$:

$$
\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

In [13]:
### Выделим test

from sklearn.model_selection import train_test_split

X = initial_data.drop('log_trip_duration', axis=1)
y = initial_data['log_trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [38]:
X.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
id,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520


In [14]:
### Применим K-Fold на оставшуюяся валидационную часть X_train, y_train

from sklearn.model_selection import KFold

splitter = KFold(n_splits=20, shuffle=True, random_state=33)

In [15]:
from sklearn.linear_model import LinearRegression

In [89]:

### Замерьте качество на кросс-валидации!
### Your code is here


losses_test = []
losses_train = []


for train_index, test_index in splitter.split(X):

    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    losses_test.append(
        
#         np.mean(
#             ((model.predict(X_test) - np.log1p(y_test)) ** 2)
#         ) ** (1/2)

        
#         np.mean(
#             (model.predict(X_test) - y_test) ** 2
#         )
        
        mean_squared_error(model.predict(X_test), y_test)
        
        
    )
    
#     losses_train.append(
#         np.mean(
#             ((model.predict(X_train) - np.log1p(y_train)) ** 2) 
#         ) ** (1/2)
#     )


In [95]:
losses_test

[0.6019567970930271,
 0.6746059226537704,
 0.6120940732083782,
 0.6030460169974731,
 0.6040891657568378,
 0.5992565884428604,
 0.6076003026124364,
 0.6210194667363349,
 0.5958806118660976,
 0.6059323879471054,
 0.6466755439920938,
 0.5979708117016365,
 0.6159370961786467,
 0.5961019850159238,
 0.5961045780952758,
 0.6693773005522549,
 0.6005708216754186,
 0.6019798203952549,
 0.6052041045049574,
 0.600513682980993]

In [92]:
from sklearn.model_selection import cross_val_score

OLS = LinearRegression()
scores = cross_val_score(OLS, X, y, cv=20)

array([ 0.04961864,  0.05233807,  0.02756176,  0.02996638,  0.04962398,
        0.0534943 ,  0.05220217,  0.05231669,  0.05055734,  0.05124283,
       -0.00254913,  0.03278122,  0.05062422,  0.02458379, -0.06685811,
        0.05258031, -0.06204282,  0.05284482,  0.05095789,  0.04623257])

In [94]:
np.mean(scores)

0.0324038460370905

In [93]:
np.mean(losses_test)


# 19.951, 19.933

# 4.463

# 4.467

# 0.6123

0.6127958539203389

In [80]:
# np.mean(losses_train)

In [81]:
### Теперь построим модель на всей тренировочной выборке
### и замерим качество на тесте!
### Your code is here

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [82]:
# train_error = np.mean(
#     ((model.predict(X_train) - np.log1p(y_train)) ** 2) ** (1/2)
# )

# train_error

In [83]:
# test_error = np.mean(
#     ((model.predict(X_test) - np.log1p(y_test)) ** 2) ) ** (1/2)

# test_error

In [84]:
np.mean(
    (model.predict(X_train) - y_train) ** 2)


0.6083896907543802

In [85]:
test_error = np.mean(
    (model.predict(X_test) - y_test) ** 2)


test_error

0.600513682980993

In [91]:
### Укажите в ответе на задание 4 среднее качество моделей на валидационных выборках
### и качество модели, обученной на полной тренировочной выборке, на тестовой выборке.
### В качестве разделителя используйте точку, ответ округлите до тысячных.

print(f'{np.mean(losses_test):.3f}, {test_error:.3f}')

# print(f'{test_error:.3f}')

# 19.951, 19.933

# 4.463, 4.462

# 4.467, 4.465

# 0.613, 0.601

0.613, 0.601


In [87]:
from sklearn.metrics import mean_squared_error

In [88]:
mean_squared_error(model.predict(X_test), y_test)

0.600513682980993

### Модель №2. Проделаем все то же самое, только для модели с более осознанными признаками, которые удалось получить ранее

In [22]:
processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [23]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


In [24]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
processed_data = processed_data.drop('trip_duration', axis=1)

In [25]:
X_2 = processed_data.drop('log_trip_duration', axis=1)
y_2 = processed_data['log_trip_duration']

In [27]:
### Важно! Когда сравниваем модели по их качеству
### на валидации и на тесте, не шаффлим данные заново!

test_indexes = X_test.index
train_indexes = X_train.index

X_train_2 = X_2[X_2.index.isin(train_indexes)]
y_train_2 = y_2[y_2.index.isin(train_indexes)]

X_test_2 = X_2[X_2.index.isin(test_indexes)]
y_test_2 = y_2[y_2.index.isin(test_indexes)]

In [28]:
### Замерьте качество на кросс-валидации!
### Your code is here

In [6]:
### Теперь построим модель и замерим качество на тесте!
### Your code is here

In [ ]:
### Укажите в ответе на задание 5 среднее качество моделей на валидационных выборках
### и качество модели, обученной на полной тренировочной выборке, на тестовой выборке.
### В качестве разделителя используйте точку, ответ округлите до тысячных.

### Какую модель среди двух стоило бы выбрать? Помогла ли нам базовая обработка признаков с первых уроков? 